# Cell-cell communication analysis

## Performing differential expression (DE) analysis of the NanoString ROIs with `limma`

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import anndata
import matplotlib as mpl
import scipy
import matplotlib.pyplot as plt

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

In [2]:
nanostring_data_path = '../KidneyDataset/'

## Read in and prepare NanoString data

In [3]:
# target QC’d, filtered, and quantile normalized (Q3) target level count data
# going to feed this into limma
Kidney_Q3Norm_TargetCountMatrix = pd.read_csv(nanostring_data_path + 'Kidney_Q3Norm_TargetCountMatrix.txt',
                                             sep='\t', index_col=0)
# ROI metadata
Kidney_Sample_Annotations = pd.read_csv(nanostring_data_path + 'Kidney_Sample_Annotations.txt',
                                             sep='\t')

In [4]:
# here in Kidney_Sample_Annotations column 'region' there is no indication of whether tubule is proximal or distal
# need to add this depending on the SegmentLabel - neg means proximal tubules,  PanCK - distal

def add_exact_region(idx, table):
    curr_SegmentLabel = table.loc[idx,'SegmentLabel']
    if curr_SegmentLabel == 'Geometric Segment':
        return('glomerulus')
    elif curr_SegmentLabel == 'neg':
        return('proximal tubule')
    elif curr_SegmentLabel == 'PanCK':
        return('distal tubule')
    else:
        return('error')
Kidney_Sample_Annotations['idx'] = Kidney_Sample_Annotations.index
Kidney_Sample_Annotations['region_exact'] = Kidney_Sample_Annotations['idx'].apply(lambda x: add_exact_region(x,Kidney_Sample_Annotations))
Kidney_Sample_Annotations.set_index('SegmentDisplayName', inplace=True)

In [5]:
# getting rid of NaNs
Kidney_Sample_Annotations['pathology'] = [elem if str(elem) != 'nan' else 'NA' for elem in Kidney_Sample_Annotations['pathology']]

In [6]:
# We will be comparing all abnormal ROIs to all healthy ROIs (acc to pathology)
# and then this 1 list of DE I will artificially copy for each of the single cell clusters
comparison_cluster = 'pathology'

In [7]:
Kidney_Q3Norm_TargetCountMatrix = Kidney_Q3Norm_TargetCountMatrix.T
# make sure indices match
Kidney_Sample_Annotations = Kidney_Sample_Annotations.loc[Kidney_Q3Norm_TargetCountMatrix.index]

In [8]:
# object with all features that passed QC (.raw.X)
adata_full = anndata.AnnData(X = Kidney_Q3Norm_TargetCountMatrix,
                             obs = Kidney_Sample_Annotations,
                             var = pd.DataFrame(index = Kidney_Q3Norm_TargetCountMatrix.columns)
                            )


_____________________________________________________________________________________________________________________________________________________________

# DE with limma

- all abnormal ROIs vs all healthy ROIs according to pathology annotation

In [9]:
# discard the ROIs that are not glomeruli (aka tubules)
adata_full = adata_full[adata_full.obs['pathology'].isin(['abnormal', 'healthy'])].copy()

In [10]:
# marker calling

t = adata_full.X.T
df = pd.DataFrame(data=t, columns= adata_full.obs.index, index=adata_full.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_full.obs.index),
                             'cell_type':[ str(i) for i in adata_full.obs['pathology']],
                             #'sample':[ str(i) for i in adata_full.obs['sample']]
                            })
meta_df.set_index('Cell', inplace=True)
meta_df.reset_index(inplace=True)

In [11]:
%load_ext rpy2.ipython

/opt/conda/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:15: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [12]:
outpath = './'

In [13]:
%%R
library(limma)
library(edgeR)

In [14]:
# because R replaces things
meta_df['Cell'] = [elem.replace(' | ','...') for elem in meta_df['Cell']]
meta_df['Cell'] = [elem.replace(' ','.') for elem in meta_df['Cell']]
#meta_df.head()

df.columns = [elem.replace(' | ','...') for elem in df.columns]

In [15]:
np.unique(meta_df['cell_type'], return_counts=True)

(array(['abnormal', 'healthy'], dtype=object), array([77, 72]))

In [16]:
case = 'abnormal'
ctrl = 'healthy'

In [17]:
%%R -i df -i meta_df -i outpath -i ctrl -i case 

library(limma)
library(edgeR)

# Format
ex_mat=as.matrix(df)
rownames(meta_df) = meta_df$Cell

# subset meta
meta_df = subset(meta_df, cell_type %in% unlist(c(ctrl, case)) )

print(unique(meta_df$cell_type))

# Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), ' shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

# Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm=T) > 0.1
ex_mat = ex_mat[ keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[ keep, ]

# Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$cell_type)
covariates = meta_df$covariate

# Extract cells in cluster and rest
cells_case = rownames(subset(meta_df, cell_type == case))
cells_ctrl = rownames(subset(meta_df, cell_type == ctrl)) # changed from control to ctrl

# build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[ cells_case ] = 'case'
cluster_type[ cells_ctrl ] = 'ctrl'

print(unique(cluster_type))

#design.matrix <- model.matrix(~ 0 + cluster_type + covariates)
design.matrix <- model.matrix(~ 0 + cluster_type)

# Now tell limma how do you want to compare (i.e. case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

# Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

# Make a dataframe containing the important data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

# Add and filter needed data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_cluster = apply(ex_mat[ results$Gene, cells_case], 1, mean)
results$AveExpr_rest = apply(ex_mat[ results$Gene, cells_ctrl], 1, mean)
results$percentExpr_cluster = apply(ex_mat[ results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_rest = apply(ex_mat[ results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)

results$AveExpr_cluster = round(results$AveExpr_cluster, 6)
results$AveExpr_rest = round(results$AveExpr_rest, 6)
results$percentExpr_cluster = round(results$percentExpr_cluster, 6)
results$percentExpr_rest = round(results$percentExpr_rest, 6)
# and store it as csv file
write.csv(results, file = paste0(outpath, case, '_vs_', ctrl, '_limma_DEGs.csv'), row.names = F, col.names = T, quote = F)

[1] "abnormal" "healthy" 


R[write to console]: 149 shared cells



[1] "case" "ctrl"


In [18]:
# let's have a look at the DEGs
DE_table = pd.read_csv('./abnormal_vs_healthy_limma_DEGs.csv', index_col=0)
DE_table

,logFC,P.Value,adj.P.Val,AveExpr_cluster,AveExpr_rest,percentExpr_cluster,percentExpr_rest
Gene,,,,,,,
HTRA1,-147.903284,3.151900e-21,5.069516e-17,160.806267,308.709550,1,1
DDN,-29.632876,2.596122e-19,1.989300e-15,24.005380,53.638256,1,1
AIF1,-34.851114,3.710458e-19,1.989300e-15,34.119944,68.971058,1,1
NEDD9,-18.092348,5.381394e-19,2.163858e-15,21.228594,39.320942,1,1
ALDOB,-103.131731,2.539504e-18,8.169075e-15,32.868548,136.000279,1,1
...,...,...,...,...,...,...,...
SLC26A11,0.000132,9.998341e-01,9.999668e-01,9.855141,9.855009,1,1
RNPEP,0.000157,9.999368e-01,9.999668e-01,38.436916,38.436759,1,1
TTC37,0.000041,9.999576e-01,9.999668e-01,15.905058,15.905017,1,1


In [19]:
# how many significant DEGs?
DE_table_significant = DE_table[DE_table['adj.P.Val'] < 0.05]
DE_table_significant

,logFC,P.Value,adj.P.Val,AveExpr_cluster,AveExpr_rest,percentExpr_cluster,percentExpr_rest
Gene,,,,,,,
HTRA1,-147.903284,3.151900e-21,5.069516e-17,160.806267,308.709550,1,1
DDN,-29.632876,2.596122e-19,1.989300e-15,24.005380,53.638256,1,1
AIF1,-34.851114,3.710458e-19,1.989300e-15,34.119944,68.971058,1,1
NEDD9,-18.092348,5.381394e-19,2.163858e-15,21.228594,39.320942,1,1
ALDOB,-103.131731,2.539504e-18,8.169075e-15,32.868548,136.000279,1,1
...,...,...,...,...,...,...,...
ANGPT2,1.529483,1.117791e-02,4.989884e-02,11.128993,9.599509,1,1
LRRC8B,1.704521,1.118756e-02,4.992562e-02,12.303394,10.598874,1,1
OR10T2,1.993797,1.119012e-02,4.992562e-02,11.400386,9.406589,1,1


# Creating a joint DE table out of limma outputs from NanoString data

Because we don't have cell type resolution here, let's copy this DE table for every cell type artificially to then intersect with the expressed L/R from Young et al. data

In [20]:
# from Young et al.
meta = pd.read_csv('./cellphonedb_meta.tsv', sep='\t')
cell_types = np.unique(meta['cell_type'])
# get rid of 'celltype_'
cell_types = [elem[9:] for elem in cell_types]
cell_types

['B cell',
 'CD8 T cell',
 'DC',
 'Distal_tubules_and_collecting_duct',
 'Endothelium',
 'Loop_of_Henle',
 'MNP',
 'Mast cell',
 'Mesangial_cells',
 'NK cell',
 'NKT cell',
 'Nephron_epithelium',
 'Neutrophil',
 'Podocytes',
 'Private',
 'Proximal_tubular_cells',
 'Th cell']

In [21]:
# let's have a look at the DEGs
DE_table = pd.read_csv('./abnormal_vs_healthy_limma_DEGs.csv')
DE_table

,Gene,logFC,P.Value,adj.P.Val,AveExpr_cluster,AveExpr_rest,percentExpr_cluster,percentExpr_rest
0,HTRA1,-147.903284,3.151900e-21,5.069516e-17,160.806267,308.709550,1,1
1,DDN,-29.632876,2.596122e-19,1.989300e-15,24.005380,53.638256,1,1
2,AIF1,-34.851114,3.710458e-19,1.989300e-15,34.119944,68.971058,1,1
3,NEDD9,-18.092348,5.381394e-19,2.163858e-15,21.228594,39.320942,1,1
4,ALDOB,-103.131731,2.539504e-18,8.169075e-15,32.868548,136.000279,1,1
...,...,...,...,...,...,...,...,...
16079,SLC26A11,0.000132,9.998341e-01,9.999668e-01,9.855141,9.855009,1,1
16080,RNPEP,0.000157,9.999368e-01,9.999668e-01,38.436916,38.436759,1,1
16081,TTC37,0.000041,9.999576e-01,9.999668e-01,15.905058,15.905017,1,1
16082,CDSN,0.000035,9.999589e-01,9.999668e-01,10.358523,10.358489,1,1


In [22]:
# artificially copying it for all the cell types
DE_tables = {}

for ct in cell_types:
    #print(ct)
    DE_tables[ct] = DE_table

B cell
CD8 T cell
DC
Distal_tubules_and_collecting_duct
Endothelium
Loop_of_Henle
MNP
Mast cell
Mesangial_cells
NK cell
NKT cell
Nephron_epithelium
Neutrophil
Podocytes
Private
Proximal_tubular_cells
Th cell


In [23]:
DE_table

,Gene,logFC,P.Value,adj.P.Val,AveExpr_cluster,AveExpr_rest,percentExpr_cluster,percentExpr_rest
0,HTRA1,-147.903284,3.151900e-21,5.069516e-17,160.806267,308.709550,1,1
1,DDN,-29.632876,2.596122e-19,1.989300e-15,24.005380,53.638256,1,1
2,AIF1,-34.851114,3.710458e-19,1.989300e-15,34.119944,68.971058,1,1
3,NEDD9,-18.092348,5.381394e-19,2.163858e-15,21.228594,39.320942,1,1
4,ALDOB,-103.131731,2.539504e-18,8.169075e-15,32.868548,136.000279,1,1
...,...,...,...,...,...,...,...,...
16079,SLC26A11,0.000132,9.998341e-01,9.999668e-01,9.855141,9.855009,1,1
16080,RNPEP,0.000157,9.999368e-01,9.999668e-01,38.436916,38.436759,1,1
16081,TTC37,0.000041,9.999576e-01,9.999668e-01,15.905058,15.905017,1,1
16082,CDSN,0.000035,9.999589e-01,9.999668e-01,10.358523,10.358489,1,1


In [24]:
cluster_list = []

for ct in cell_types:
    #print(ct)
    cluster_list.append([ct]*len(DE_table))

cluster_list = [item for sublist in cluster_list for item in sublist]

B cell
CD8 T cell
DC
Distal_tubules_and_collecting_duct
Endothelium
Loop_of_Henle
MNP
Mast cell
Mesangial_cells
NK cell
NKT cell
Nephron_epithelium
Neutrophil
Podocytes
Private
Proximal_tubular_cells
Th cell


In [25]:
# without any filtering
joint_DE_table = pd.concat(DE_tables.values())

In [26]:
joint_DE_table['cluster'] = cluster_list

In [27]:
joint_DE_table

,Gene,logFC,P.Value,adj.P.Val,AveExpr_cluster,AveExpr_rest,percentExpr_cluster,percentExpr_rest,cluster
0,HTRA1,-147.903284,3.151900e-21,5.069516e-17,160.806267,308.709550,1,1,B cell
1,DDN,-29.632876,2.596122e-19,1.989300e-15,24.005380,53.638256,1,1,B cell
2,AIF1,-34.851114,3.710458e-19,1.989300e-15,34.119944,68.971058,1,1,B cell
3,NEDD9,-18.092348,5.381394e-19,2.163858e-15,21.228594,39.320942,1,1,B cell
4,ALDOB,-103.131731,2.539504e-18,8.169075e-15,32.868548,136.000279,1,1,B cell
...,...,...,...,...,...,...,...,...,...
16079,SLC26A11,0.000132,9.998341e-01,9.999668e-01,9.855141,9.855009,1,1,Th cell
16080,RNPEP,0.000157,9.999368e-01,9.999668e-01,38.436916,38.436759,1,1,Th cell
16081,TTC37,0.000041,9.999576e-01,9.999668e-01,15.905058,15.905017,1,1,Th cell
16082,CDSN,0.000035,9.999589e-01,9.999668e-01,10.358523,10.358489,1,1,Th cell


In [28]:
joint_DE_table.to_csv('./joint_DEGs_list_all_cell_types_for_cellphone.csv')